In [1]:
import dask
import socket
from dask_jobqueue import SLURMCluster
from  distributed import Client
import subprocess


In [29]:
!conda list

# packages in environment at /home/arjunp/.conda/envs/r_env:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
_r-mutex                  1.0.0               anacondar_1  
_sysroot_linux-64_curr_repodata_hack 3                   haa98f57_10  
anyio                     4.2.0            py39h06a4308_0  
argon2-cffi               21.3.0             pyhd3eb1b0_0  
argon2-cffi-bindings      21.2.0           py39h7f8727e_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
async-lru                 2.0.4            py39h06a4308_0  
attrs                     23.1.0           py39h06a4308_0  
babel                     2.11.0           py39h06a4308_0  
backcall                  0.2.0              pyhd3eb1b0_0  
beautifulsoup4            4.12.3           py39h06a4308_0  
binutils_impl_linux-64    2.38                 h2a08ee3_1  
binutils_linux-64  

In [2]:
!hostname

node072


In [8]:
#!pip install dask-jobqueue

Defaulting to user installation because normal site-packages is not writeable
  Using cached dask_jobqueue-0.8.5-py2.py3-none-any.whl.metadata (1.6 kB)
Using cached dask_jobqueue-0.8.5-py2.py3-none-any.whl (47 kB)


In [39]:
cluster2 = SLURMCluster(cores=8,
                       processes=4,
                       memory="16GB",
                       account="cpl",
                       walltime="01:00:00",
                       queue="normal",
                       job_script_prologue  =[
                        'source /etc/profile.d/modules.sh' ,
                        'module load openmind8/anaconda/3-2023.09-0',
                        'module load openmind/gcc/11.1.0',
                        'module load openmind/ffmpeg/20160310' , 'source ~/.bashrc',
                        'conda activate codebase3',
                        'cd /om/user/arjunp/ashraf_repo/whisper-diarization',
                        'conda list'
                                              ],
                       job_extra=['--gres=gpu:QUADRORTX6000:1'] 
                       )
cluster2.scale(1)
print(cluster2.job_script())
client2 = Client(cluster2)

/home/arjunp/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/cm/shared/openmind8/anaconda/3-2023.09-0/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42599 instead
  warnings.warn(


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p normal
#SBATCH -A cpl
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=15G
#SBATCH -t 01:00:00
#SBATCH --gres=gpu:QUADRORTX6000:1
source /etc/profile.d/modules.sh
module load openmind8/anaconda/3-2023.09-0
module load openmind/gcc/11.1.0
module load openmind/ffmpeg/20160310
source ~/.bashrc
conda activate codebase3
cd /om/user/arjunp/ashraf_repo/whisper-diarization
conda list
/cm/shared/openmind8/anaconda/3-2023.09-0/bin/python -m distributed.cli.dask_worker tcp://172.16.20.80:46205 --name dummy-name --nthreads 2 --memory-limit 3.73GiB --nworkers 4 --nanny --death-timeout 60



In [33]:
!squeue -u arjunp

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          37425834    normal dask-wor   arjunp  R       0:04      1 node079
          37419856    normal     bash   arjunp  R    1:34:46      1 node098


In [27]:
#!scancel 37425828

In [40]:
def run_cb3_script(**kwargs):
     
    command = ['python', '/om/user/arjunp/ashraf_repo/whisper-diarization/diarize.py','-a','/om/user/arjunp/first_min_030327.wav']  
    
    result = subprocess.run(command, capture_output=True, text=True)
     
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    return result

In [41]:
future2 = client2.submit(run_cb3_script)  
result2 = future2.result()

In [42]:
print(result2.stdout)

Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /weka/scratch/weka/cpl/arjunp/ashraf_repo/whisper-diarization/temp_outputs/htdemucs
Separating track /om/user/arjunp/first_min_030327.wav
GPU Name: Quadro RTX 6000
CUDA Compute Capability: (7, 5)
Your GPU supports efficient float16 computation.
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2+cu121. Bad things might happen unless you revert torch to 1.x.
[NeMo I 2024-07-24 13:49:03 msdd_models:1092] Loading pretrained diar_msdd_telephonic model from NGC
[NeMo I 2024-07-24 13:49:03 cloud:58] Found existing object /home/arjunp/.cache/torch/NeMo/NeMo_1.20.0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-07-24 13:49:03 cloud:64] Re-using file from: /home/arjunp/.cache/torch/NeMo/NeMo_1.20.0/di

In [43]:
result2.stderr

'[NeMo W 2024-07-24 13:48:30 nemo_logging:349] /home/arjunp/.conda/envs/codebase3/lib/python3.8/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.\n      torchaudio.set_audio_backend("soundfile")\n    \ntorchvision is not available - cannot save figures\n/home/arjunp/.conda/envs/codebase3/lib/python3.8/site-packages/demucs/api.py:27: UserWarning: TorchAudio\'s global backend is now deprecated. Please enable distpatcher by setting `TORCHAUDIO_USE_BACKEND_DISPATCHER=1`, and specify backend when calling load/info/save function.\n  import torchaudio as ta\n\n  0%|                                                                                  | 0.0/64.35 [00:00<?, ?seconds/s]\n  9%|██████▋                                                                  | 5.85/64.35 [00:02<00:29,  1.99seconds/s]\n 18%|█████████████▎                            

In [23]:
!conda  info --envs

# conda environments:
#
base                     /cm/shared/openmind8/anaconda/3-2023.09-0
jax                      /cm/shared/openmind8/anaconda/3-2023.09-0/envs/jax
nemo2                    /cm/shared/openmind8/anaconda/3-2023.09-0/envs/nemo2
test                     /cm/shared/openmind8/anaconda/3-2023.09-0/envs/test
test1                    /cm/shared/openmind8/anaconda/3-2023.09-0/envs/test1
codebase3                /home/arjunp/.conda/envs/codebase3
r_env                 *  /home/arjunp/.conda/envs/r_env
torch_gpu                /home/arjunp/.conda/envs/torch_gpu



# Trying to set up CB1

In [1]:
!pwd

/om/user/arjunp


In [3]:
cluster4 = SLURMCluster(cores=4,
                       processes=2,
                       memory="16GB",
                       account="cpl",
                       walltime="01:00:00",
                       queue="normal",
                       job_script_prologue  =[
                        'source /etc/profile.d/modules.sh' ,
                        'module load openmind8/anaconda/3-2023.09-0',
                        'module load openmind/gcc/11.1.0',
                        'module load openmind/ffmpeg/20160310' , 
                        'source ~/.bashrc',
                        'export MKL_THREADING_LAYER=GNU',
                        'conda activate torch_gpu'
                                              ],
                       job_extra=['--gres=gpu:QUADRORTX6000:1'] 
                       )
cluster4.scale(1)
#print(cluster2.job_script())
#client3 = Client(cluster3)
client4 = Client(cluster4)

/home/arjunp/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/arjunp/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [65]:
!squeue -u arjunp

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          37435051    normal dask-wor   arjunp  R       0:03      1 node078
          37435035    normal dask-wor   arjunp  R      12:03      1 node092
          37434728    normal     bash   arjunp  R    1:11:38      1 node080


In [33]:
!scancel 37434780

In [4]:
def run_cb1_script(**kwargs):
     
    command = ['python', 'yinruiqing_trial.py', 'first_min_030327.wav']  
    
    result = subprocess.run(command, capture_output=True, text=True)
     
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    return result

In [5]:
future4 = client4.submit(run_cb1_script)  
result4 = future4.result()

In [50]:
print(result3.stdout)

Running CB1: yinruiqing
Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu118. Bad things might happen unless you revert torch to 1.x.
cuda
0.00 2.00 SPEAKER_00  You have to press down harder though, honey.
2.00 4.00 SPEAKER_00  Whoa!
4.00 6.00 SPEAKER_00  Look!
6.00 8.00 SPEAKER_00  Look at this! This is cute.
8.00 10.00 SPEAKER_00  May I have a piece of paper?
10.00 12.00 SPEAKER_00  This one, mom.
12.00 14.00 SPEAKER_00  This one's for me?
14.00 16.00 SPEAKER_00  This one's for me.
16.00 18.00 SPEAKER_00  Alright, you have to press down like this. Watch.
18.00 20.00 SPEAKER_00  To make the colors show.
20.00 34.00 SPEAKER_00  A L E X Spouse X Spouse what?
34.00 42.00 SPEAKER_00  X He's a snow Wolf Hey, mom.
42.00 44.00 SPEAKER_00  I see you.
44.00 46.00 SPEAKER_00  Can I play with this?
46.00 48.00 SPEAKER_00  Which one?
48.00 50.00 SPEAKER_00  I se

In [7]:
print(result4.stdout)

Running CB1: yinruiqing
Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu118. Bad things might happen unless you revert torch to 1.x.
cuda
0.00 2.00 SPEAKER_00  You have to press down harder though honey.
2.00 4.00 SPEAKER_00  Oh my...
4.00 6.00 SPEAKER_00  Look!
6.00 8.00 SPEAKER_00  Look at this! This is cute.
8.00 10.00 SPEAKER_00  May I have a piece of paper?
10.00 12.00 SPEAKER_00  Oh this one mom. This one's for me?
12.00 14.00 SPEAKER_00  This one's for me. Alright, may I have a color paper?
14.00 16.00 SPEAKER_00  Oh this one! Alright, you have to press down like this.
16.00 18.00 SPEAKER_00  Watch.
18.00 20.00 SPEAKER_00  To make the colors show.
20.00 34.00 SPEAKER_00  A L E X Spouse X Spouse what?
34.00 42.00 SPEAKER_00  X He's a snow Wolf Hey mom!
42.00 44.00 SPEAKER_00  I see you.
44.00 46.00 SPEAKER_00  Can I play with this?
46.00 48.00 

## Trying CB2 with the same env

In [18]:
cluster4 = SLURMCluster(cores=4,
                       processes=2,
                       memory="16GB",
                       account="cpl",
                       walltime="01:00:00",
                       queue="normal",
                       job_script_prologue  =[
                        'source /etc/profile.d/modules.sh' ,
                        'module load openmind8/anaconda/3-2023.09-0',
                        'module load openmind/gcc/11.1.0',
                        'module load openmind/ffmpeg/20160310' , 
                        'source ~/.bashrc',
                        'export MKL_THREADING_LAYER=GNU',
                        'conda activate torch_gpu'
                                              ],
                       job_extra=['--gres=gpu:QUADRORTX6000:1'] 
                       )
cluster4.scale(1)
#print(cluster2.job_script())
#client3 = Client(cluster3)
client4 = Client(cluster4)

/home/arjunp/.local/lib/python3.11/site-packages/dask_jobqueue/core.py:266: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/cm/shared/openmind8/anaconda/3-2023.09-0/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42847 instead
  warnings.warn(


In [19]:
def run_cb2_script(**kwargs):
     
    command = ['python', 'speechbox_trial.py', 'first_min_030327.wav']  
    
    result = subprocess.run(command, capture_output=True, text=True)
     
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    return result

In [20]:
!squeue -u arjunp

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          37441287    normal dask-wor   arjunp  R       0:09      1 node088
          37441265    normal     bash   arjunp  R      25:40      1 node072
          37441264    normal     bash   arjunp  R      27:51      1 node097


In [21]:
future5 = client4.submit(run_cb2_script)  
result5 = future5.result()

In [22]:
print(result5.stdout)

Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu118. Bad things might happen unless you revert torch to 1.x.
Running CB2: speechbox
[{'speaker': 'SPEAKER_00', 'text': ' You have to press down like this, watch. To make the colors show. A L E E X Spouse A Ex-spouse. Ex-spouse. Ex-spouse. Yes. Ex-spouse. Yes. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Ex-spouse. Hey, Mom, first you eat, I see you. For me, for me, what is this? Which one?', 'timestamp': (0.0, 70.0)}, {'speaker': 'SPEAKER_01', 'text': ' I see you, I see you, I see you, I see you.', 'timestamp': (70.0, 74.0)}, {'speaker': 'SPEAKER_00', 'text': ' Mommy, what is this?', 'timestamp': (74.0, 77.0)}, {'speaker': 'SPEAKER_01', 'text': ' M-m-m-m-m-m.', 'timestamp': (79.0, 81.0)}, {'speaker': 'SPEAK

In [16]:
!squeue -u arjunp

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          37441271    normal dask-wor   arjunp  R      18:57      1 node088
          37441265    normal     bash   arjunp  R      25:02      1 node072
          37441264    normal     bash   arjunp  R      27:13      1 node097
